In [5]:
import numpy as np
import pandas as pd 

from torchtext.legacy import data
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB
from torchtext.legacy.data import Field, LabelField, BucketIterator

In [6]:

import os
for dirname, _, filenames in os.walk('stanfordSentimentTreebank'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

stanfordSentimentTreebank/SOStr.txt
stanfordSentimentTreebank/sentiment_labels.txt
stanfordSentimentTreebank/README.txt
stanfordSentimentTreebank/original_rt_snippets.txt
stanfordSentimentTreebank/datasetSplit.txt
stanfordSentimentTreebank/dictionary.txt
stanfordSentimentTreebank/STree.txt
stanfordSentimentTreebank/datasetSentences.txt
stanfordSentimentTreebank/.ipynb_checkpoints/README-checkpoint.txt
stanfordSentimentTreebank/.ipynb_checkpoints/datasetSplit-checkpoint.txt
stanfordSentimentTreebank/.ipynb_checkpoints/datasetSentences-checkpoint.txt
stanfordSentimentTreebank/.ipynb_checkpoints/sentiment_labels-checkpoint.txt


In [7]:

def get_merged_dataset(sst_dir,split_idx):
    sentiment_labels = pd.read_csv(os.path.join(sst_dir, "sentiment_labels.txt"), sep="|")
    sentence_ids = pd.read_csv(os.path.join(sst_dir, "datasetSentences.txt"), sep="\t")
    dictionary = pd.read_csv(os.path.join(sst_dir, "dictionary.txt"), sep="|", names=['phrase', 'phrase ids'])
    train_test_split = pd.read_csv(os.path.join(sst_dir, "datasetSplit.txt"))
    sentence_phrase_merge = pd.merge(sentence_ids, dictionary, left_on='sentence', right_on='phrase')
    sentence_phrase_split = pd.merge(sentence_phrase_merge, train_test_split, on='sentence_index')
    final = pd.merge(sentence_phrase_split, sentiment_labels, on='phrase ids').sample(frac=1)
    
#     return final[final["splitset_label"] == split_idx].reset_index()
    return final


def discretize_label(label):
    if label <= 0.2: return 0
    if label <= 0.4: return 1
    if label <= 0.6: return 2
    if label <= 0.8: return 3
    return 4

In [8]:

def get_data(sst_dir,split_idx):
    df = get_merged_dataset(sst_dir,split_idx)
    df['label'] = df['sentiment values'].apply(discretize_label)
    df = df[['sentence','label']]
    TEXT = data.Field (sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
    LABEL = data.LabelField (tokenize ='spacy', is_target=True, batch_first =True, sequential =False)
    fields = [('Text', TEXT),('labels',LABEL)]
    example = [data.Example.fromlist([df.sentence[i],df.label[i]], fields) for i in range(df.shape[0])]
    
    return data.Dataset(example, fields)
    

In [9]:
sst_dir= 'stanfordSentimentTreebank/'
split_idx = 1

In [10]:
df = get_data(sst_dir,split_idx)

/home/jd/anaconda3/envs/eva/lib/python3.8/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [11]:
type(df)

torchtext.legacy.data.dataset.Dataset

In [14]:
len(df)

11286

In [ ]:
TEXT.build_vocab(df)
LABEL.build_vocab(df)